In [ ]:
import os
import argparse
from openai import OpenAI
import tiktoken

import json
import time
from tqdm.auto import tqdm
from nltk.corpus import stopwords

In [ ]:
OPENAI_API_KEY = ''

client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

In [ ]:
data_path = '../data/analogy'
test_path = os.path.join(data_path, 'MC_test.json')
test_data = json.load(open(test_path, 'r'))

In [ ]:
# Filter out invalid examples (whose answers are not in the ChatGPT's vocabulary)
uids = []
prompts = []

for example in tqdm(test_data):
    uid = example['uid']
    prompt = example['input']
    prompt = f' {prompt} OUTPUT ONLY LETTER.'

    if True: # there is no filtering rule for this dataset
        uids.append(uid)
        prompts.append(prompt)

In [ ]:
# Run completions with API and store the results
raw_predictions = []
raw_predictions_remove_stopwords = []

batch_size = 1 # chatgpt does not support batched completion for now
for i in tqdm(range(0, len(prompts), batch_size)):
    uid_batch = uids[i:i+batch_size]
    prompt_batch = prompts[i:i+batch_size]
    messages = []
    for prompt in prompt_batch:
        messages.append({"role": "user", "content": prompt})

In [ ]:
# target_model = 'gpt-3.5-turbo-0125'
# target_model = 'gpt-4o-2024-08-06'

target_model = 'gpt-4o-2024-08-06'

In [ ]:
# Run completions with API and store the results
raw_predictions = []
raw_predictions_remove_stopwords = []

batch_size = 1 # chatgpt does not support batched completion for now
for i in tqdm(range(0, len(prompts), batch_size)):
    uid_batch = uids[i:i+batch_size]
    prompt_batch = prompts[i:i+batch_size]
    messages = []
    for prompt in prompt_batch:
        messages.append({"role": "user", "content": prompt})

    while True:
        try:
            # responses = client.chat.completions.create(
			# 	model=args.target_model,
			# 	messages=messages,
			# 	max_tokens=1,
			# 	temperature=0,
            #     logprobs=True,
            #     top_logprobs=20,
			# )

            responses_remove_stopwords = client.chat.completions.create(
				model=target_model,
				messages=messages,
				max_tokens=1,
				temperature=0,
                logprobs=True,
                top_logprobs=20,
			)
            break
        except Exception as e:
            print('Error!', e)
            time.sleep(3)

    # for uid, response in zip(uid_batch, responses.choices):
    #     raw_predictions.append({"uid": uid, "response": response})
    # for uid, response_remove_stopwords in zip(uid_batch, responses_remove_stopwords.choices):
    #     raw_predictions_remove_stopwords.append({"uid": uid, "response": response_remove_stopwords})
    uid = uid_batch[0]
    logprobs_remove_stopwords = responses_remove_stopwords.choices[0].logprobs.content[0].top_logprobs
    top_k_tokens_remove_stopwords, top_k_logprobs_remove_stopwords = [], []
    for logprob in logprobs_remove_stopwords:
        top_k_tokens_remove_stopwords.append(logprob.token)
        top_k_logprobs_remove_stopwords.append(logprob.logprob)
    raw_predictions_remove_stopwords.append({"uid": uid, "top_k_tokens_remove_stopwords": top_k_tokens_remove_stopwords, "top_k_logprobs_remove_stopwords": top_k_logprobs_remove_stopwords})

# Write the results
out_path = os.path.join('results', target_model)
os.makedirs(out_path, exist_ok=True)

# with open(os.path.join(out_path, f'raw_pred_{args.dataset_name}_{args.dataset_type}.json'), 'w') as fout:
#     json.dump(raw_predictions, fout)

with open(os.path.join(out_path, f'raw_pred_analogy_remove_stopwords.json'), 'w') as fout:
    json.dump(raw_predictions_remove_stopwords, fout)